# Simulation
With R kernel

#### Further info
Models usable with train() from caret: <br>
https://topepo.github.io/caret/train-models-by-tag.html#Model_Tree <br>

## Packages

In [12]:
.libPaths("/user/emma.foessing01/u11969/R_libs")
Sys.setenv("PKG_CXXFLAGS"="-std=c++14")

install.packages("synthpop")

# Load libraries
list_of_packages <- c(
  "synthpop", "insight", "party", "haven", "dplyr", "rpart", "rpart.plot",
  "randomForest", "pROC", "caret", "pracma", "here", "Hmisc", "purrr",
  "ranger", "bnlearn", "arulesCBA", "network", "igraph", "xgboost",
  "data.table", "RSNNS"
)
# Function to check and install packages
install_if_missing <- function(p) {
  if (!requireNamespace(p, quietly = TRUE)) {
    install.packages(p, dependencies = TRUE)
  }
  library(p, character.only = TRUE)
}

# Install and load all required packages
lapply(list_of_packages, install_if_missing)

Lade n"otiges Paket: Hmisc

Warning message:
"Paket 'Hmisc' wurde unter R Version 4.3.2 erstellt"

Attache Paket: 'Hmisc'


Das folgende Objekt ist maskiert 'package:pracma':

    ceil


Die folgenden Objekte sind maskiert von 'package:dplyr':

    src, summarize


Die folgenden Objekte sind maskiert von 'package:base':

    format.pval, units




[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

## Data

In [ ]:
# set path
############## adjust to correct directory!
directory <- "/user/emma.foessing01/u11969/Master-Thesis"

In [3]:
load(file = (paste0(directory, "/cpspop.RData")))
adult <- read.csv(file = (directory,"/adult_preprocessed.csv")))
# delete NAs
adult[adult == "?"] <- NA
adult <- na.omit(adult)

adult$workclass <- as.factor(adult$workclass)
adult$education <- as.factor(adult$education)
adult$marital_status <- as.factor(adult$marital_status)
adult$relationship <- as.factor(adult$relationship)
adult$race <- as.factor(adult$race)
adult$sex <- as.factor(adult$sex)
adult$native_country <- as.factor(adult$native_country)
adult$income <- as.factor(adult$income)

Ein Datensatz zu genieren (m = 1) ist ausreichend, da ich keine Varianzanalyse machen werde. Damit die Ergebnisse nicht von einem zufälligen Prozess abhängen ist es sinnvoll über ein paar runs Mittelwerte zu bilden (50–100)

## Functions

### Evaluation Functions

In [58]:
## Calculate evaluation metrics for continuous targets
evaluation_metrics_cont <- function(predictions, test_set){
  MAE <- mean(abs(predictions - test_set$income))
  MSE <- mean((predictions - test_set$income)^2)
  RMSE <- sqrt(MSE)

  SS_res <- sum((test_set$income - predictions)^2)
  SS_tot <- sum((test_set$income - mean(test_set$income))^2)
  R_squared <- 1 - (SS_res / SS_tot)

  MAPE <- mean(abs((test_set$income - predictions) / test_set$income)) * 100

  # Create the dataframe
  metrics_df <- data.frame(
        MAE = MAE, 
        MSE = MSE, 
        RMSE = RMSE,
        R_squared = R_squared, 
        MAPE = MAPE)

  return(metrics_df)
}

In [59]:
## Calculate evaluation metrics for factored targets
evaluation_metrics_factor <- function(predictions, test_set){
    # confusion matrix for the prediction on original data
    cm <- confusionMatrix(predictions, test_set$income,
                mode = "everything")

    # saving evaluation metrics
    accuracy <- cm$overall['Accuracy']
    f1 <- cm$byClass['F1']
    sens <- cm$byClass['Sensitivity']
    spec <- cm$byClass['Specificity']

    # Create the dataframe
    metrics_df <- data.frame(
        Accuracy = accuracy, 
        F1 = f1, 
        Sensitivity = sens, 
        Specificity = spec
    )
    
    return(metrics_df)
}

### Prediction Functions

In [ ]:
# just the prediction
cart_pred <- function(data, outer_folds, cp_steps, inner_folds){############adjust##############
    # adjust evaluation metric to fit both numeric and factored targets
    summaryFunctionType <- if (is.numeric(data$income)) defaultSummary else multiClassSummary
    # metric: train() uses per default RSME and Accuracy for numeric and factored targets

    #  set control args
    outer_control <- trainControl(method = "cv", number = outer_folds,
                                  summaryFunction = summaryFunctionType,
                                  verboseIter = FALSE,
                                  allowParallel = TRUE)
        
    inner_control <- trainControl(method = "cv", number = inner_folds, 
                                  summaryFunction = summaryFunctionType,
                                  verboseIter = FALSE,
                                  allowParallel = TRUE)

    # Define the grid for hyperparameter tuning
    complexity <- 10^seq(log10(0.0001), log10(0.01), length.out = cp_steps)############adjust##############

    # Create grid
    tunegrid <- expand.grid(cp = complexity)############adjust##############

    # Initialize variables to store results
    outer_results <- list()

    outer_cv_folds = createFolds(data$income, k = outer_folds)
    
    # Outer loop: Cross-validation for model evaluation
    for (i in seq_along(outer_folds)) {
        
        # Split data into outer folds
        outer_test_index = outer_cv_folds[[i]]
        outer_testData = data[outer_test_index,]
        outer_trainData  = data[-outer_test_index,]
        
        # Hyperparameter tuning using inner CV
        # No need for inner loop because "train" does k-fold CV already
        model <- caret::train(income ~ ., 
                        data = outer_trainData, 
                        method = "rpart", ############adjust##############
                        tuneGrid = tunegrid, 
                        trControl = inner_control,
                        control = rpart.control(maxsurrogate = 0, maxcompete = 1) ############adjust##############
                        )#,
                        #metric = metricType)
            

        # Store the best hyperparameters
        best_hyperparameters <- model$bestTune

        # Train the final model on the outer training set with the best hyperparameters
        final_model <- caret::train(income ~ ., 
                             data = outer_trainData, 
                             method = "rpart",############adjust##############
                             trControl = outer_control, 
                             tuneGrid = best_hyperparameters)

        # Testing the final model on the outer test set
        predictions <- predict(final_model, newdata = outer_testData)
        
        if (is.numeric(data$income)) {
            eval <- postResample(predictions, outer_testData$income) # postResample is a useful caret function
        } else if (is.factor(data$income)) {
            eval <- confusionMatrix(predictions, outer_testData$income)
        } else {
            stop("The predicted target has to be numeric or factor.")
        }

        # Store the evaluation metrics for this outer fold
        outer_results[[i]] <- eval
    }

    # Average the evaluation metrics over the outer folds
    eval_avg_outer_fold <- mean(unlist(outer_results)) # Calculate the mean performance over all outer folds

    # Return the average evaluation metrics
    return(eval_avg_outer_fold)
}

In [ ]:
# just the prediction
rf_pred <- function(data, outer_folds, mtry_steps, ntree_steps, inner_folds){############adjust##############
    # adjust evaluation metric to fit both numeric and factored targets
    summaryFunctionType <- if (is.numeric(data$income)) defaultSummary else multiClassSummary
    # metric: train() uses per default RSME and Accuracy for numeric and factored targets

    #  set control args
    outer_control <- trainControl(method = "cv", number = outer_folds,
                                  summaryFunction = summaryFunctionType,
                                  verboseIter = FALSE,
                                  allowParallel = TRUE)
        
    inner_control <- trainControl(method = "cv", number = inner_folds, 
                                  summaryFunction = summaryFunctionType,
                                  verboseIter = FALSE,
                                  allowParallel = TRUE)

    # Define the parameter grid for tuning
    splitrule_value <- if (is.numeric(data$income)) "variance" else "gini"
    
    tunegrid <- expand.grid(mtry = seq(2, ncol(data) - 1, length.out = mtry_steps),
    splitrule = splitrule_value,
    min.node.size = 5)
    #ntree_steps noch einbauen?

    # Initialize variables to store results
    outer_results <- list()

    outer_cv_folds = createFolds(data$income, k = outer_folds)
    
    # Outer loop: Cross-validation for model evaluation
    for (i in seq_along(outer_folds)) {
        
        # Split data into outer folds
        outer_test_index = outer_cv_folds[[i]]
        outer_testData = data[outer_test_index,]
        outer_trainData  = data[-outer_test_index,]
        
        # Hyperparameter tuning using inner CV
        # No need for inner loop because "train" does k-fold CV already
        model <- caret::train(income ~ ., 
                        data = outer_trainData, 
                        method = "ranger", ############adjust##############
                        tuneGrid = tunegrid, 
                        trControl = inner_control)#,
                        #metric = metricType)
            

        # Store the best hyperparameters
        best_hyperparameters <- model$bestTune

        # Train the final model on the outer training set with the best hyperparameters
        final_model <- caret::train(income ~ ., 
                             data = outer_trainData, 
                             method = "ranger",############adjust##############
                             trControl = outer_control, 
                             tuneGrid = best_hyperparameters)

        # Testing the final model on the outer test set
        predictions <- predict(final_model, newdata = outer_testData)
        
        if (is.numeric(data$income)) {
            eval <- postResample(predictions, outer_testData$income) # postResample is a useful caret function
        } else if (is.factor(data$income)) {
            eval <- confusionMatrix(predictions, outer_testData$income)
        } else {
            stop("The predicted target has to be numeric or factor.")
        }

        # Store the evaluation metrics for this outer fold
        outer_results[[i]] <- eval
    }

    # Average the evaluation metrics over the outer folds
    eval_avg_outer_fold <- mean(unlist(outer_results)) # Calculate the mean performance over all outer folds

    # Return the average evaluation metrics
    return(eval_avg_outer_fold)
}

In [ ]:
#boost_pred <-

In [ ]:
#bn_pred <-

In [ ]:
#mlp_pred <-

## For original data

In [ ]:
simulation <- function(data, nrun = 10, outer_folds, inner_folds, mtry_steps = 10, ntree_steps= 10, cp_steps = 10){
    
    # create empty list to store evaluation dataframes
    eval_list <- list()

    # set inital seed
    s <- 1234
    for (i in 1:nrun){
        # vary seed with each run
        s <- s + 1

        # create synthetic data
        # data <- gen_data()

        # prediction model with nested CV and grid search
        CART_eval <- cart_pred(data, outer_folds, cp_steps, inner_folds)
        RF_eval <- rf_pred(data, outer_folds, mtry_steps, ntree_steps, inner_folds)
        #Boost_eval <-
        #BN_eval <-
        #MLP_eval <-

        # bind results 
        eval <- rbind(CART_eval = CART_eval, RF_eval = RF_eval)#, Boost_eval = Boost_eval, BN_eval = BN_eval, MLP_eval = MLP_eval)

        # ich glaube wenn es so verschachtelt ist und ich eine Liste und noch eine Liste habe, müsste ich es anders machen
        # am besten wäre ein Dataframe und für jede Model-Art eine Zeile
        eval_list[[i]] <- eval
        print(c("run", i, "completed"))
        }

    # average over all runs
    sum_df <- Reduce(function(x, y) Map(`+`, x, y), eval_list)
    eval_avg <- lapply(sum_df, function(col) col / length(eval_list))

    # Convert the list back to a dataframe
    # Store row names
    rownames <- row.names(eval_list[[1]])

    # Convert the list back to a dataframe
    eval_avg <- as.data.frame(eval_avg)

    # Set back the row names
    row.names(eval_avg) <- rownames
    
    # returns
    results <- list(eval_avg = eval_avg)
    return(results)
}

## For synthetic data

old

simulation <- function(data, nrun = 10, k_fold = 10, steps = 10){
    # create array to save the synthetic data
    # syn_data <- array(data = NA, dim = c(k, ncol(data), nrun)) # not necessary, but in case it's wanted
    # create empty vector to safe trees
    tree <- list()
    # create vector to safe loss
    loss <- rep(0, steps)
    # create empty array to store cp values chosen
    cp_val <- rep(0, nrun)
    # create empty list to store evaluation dataframes
    eval_list <- list()

    # set the complexity parameters for trees
    complexity <- 10^seq(log10(0.0001), log10(0.01), length.out = steps)

    # for loss-calculation factored variables need to be converted to numeric
    if (is.factor(data$income)) {
        data$income <- as.factor(as.numeric(data$income == ">50K"))
    }

    # set inital seed
    s <- 1234
    for (i in 1:nrun){
        # vary seed with each run
        s <- s + 1

        # generate synthetic data
        gen_data <- syn(data = data, k = nrow(data), seed = s)
        # save the data from each run if wanted
        # syn_data[,,i] <- as.matrix(gen_data$syn)

        # Randomly split the data set into k-subsets (or k-fold)
        data_syn <- as.data.frame(gen_data$syn) # try without this?
        datalist_syn <- split(gen_data$syn, sample(1:k_fold, nrow(gen_data$syn), replace=T)) #list of k same-sized elements that are slices of the data
        
        # leave-one-out CV for prediction
        for (j in 1:k_fold) {
            # split data in k folds
            data_val <- datalist_syn[[j]]               # j-th of the k folds, validation set
            data_train <- bind_rows(datalist_syn[-j])   #rest of the data without j-th of the k folds, training set

            # optional parameters to prevent overfitting: minbucket, minsplit, maxdepth
            for (l in 1:length(complexity)){
                # create income prediction tree with train data
                tree[[l]] <- rpart(income ~ ., data = data_train, cp = complexity[l], control = rpart.control(maxsurrogate = 0, maxcompete = 1))
                # Predict on the validation set
                predictions <- predict(tree[[l]], data_val)

                # safe some loss information and sum over the k-fold loops
                if (is.numeric(data$income)) {
                    # Mean Squared Error
                    loss[l] <- loss[l] + mean((predictions - data_val$income)^2)
                    }
                else if (is.factor(data$income)) {
                    # Cross-Entropy Loss
                    epsilon <- 1e-15  # to prevent log(0) which is undefined
                    predicted_probs <- pmax(pmin(predictions[,2], 1 - epsilon), epsilon)
                    n <- length(predicted_probs)
                    loss[l] <- loss[l] + (-sum(as.numeric(data_val$income) * log(predicted_probs) + (1 - as.numeric(data_val$income)) * log(1 - predicted_probs)) / n)
                    }
                else {
                    break("The predicted target has to be numeric or factor.")
                    }
                }
            }
        # for which cp value was the loss the smallest
        min_loss <- which.min(loss)
        print(min_loss)
        cp_val[i] <- c(complexity[min_loss])

        tree_s <- build_tree(data = gen_data$syn, cp = cp_val[i])
    
        # evaluation metrics
        if (is.numeric(data$income)) {
            eval <- as.data.frame(evaluation_metrics_cont(tree_s$predictions, tree_s$test_set))
            }
        else if (is.factor(data$income)) {
            eval <- as.data.frame(evaluation_metrics_factor(tree_s$predictions$classes, tree_s$test_set))
            }
        else {
            break("The predicted target has to be numeric or factor.")
            }

        eval_list[[i]] <- eval
        print(c("run", i, "completed"))
        }

    # average over all runs
    sum_df <- Reduce(function(x, y) Map(`+`, x, y), eval_list)
    eval_avg <- lapply(sum_df, function(col) col / length(eval_list))

    # Convert the list back to a dataframe
    # Store row names
    rownames <- row.names(eval_list[[1]])

    # Convert the list back to a dataframe
    eval_avg <- as.data.frame(eval_avg)

    # Set back the row names
    row.names(eval_avg) <- rownames
    
    # returns
    results <- list(eval_avg = eval_avg,  cp_vals = cp_val)
    return(results)
}

In [62]:
adult_res <- simulation(data = adult, nrun = 10, outer_folds = 5, inner_folds = 5, mtry_steps = 10, ntree_steps = 10, cp_steps = 10)


Synthesis
-----------
 age workclass fnlwgt education marital_status occupation relationship race sex capital_gain
 capital_loss hours_per_week native_country income
[1] 1
[1] "run"       "1"         "completed"

Synthesis
-----------
 age workclass fnlwgt education marital_status occupation relationship race sex capital_gain
 capital_loss hours_per_week native_country income
[1] 1
[1] "run"       "2"         "completed"

Synthesis
-----------
 age workclass fnlwgt education marital_status occupation relationship race sex capital_gain
 capital_loss hours_per_week native_country income
[1] 1
[1] "run"       "3"         "completed"

Synthesis
-----------
 age workclass fnlwgt education marital_status occupation relationship race sex capital_gain
 capital_loss hours_per_week native_country income
[1] 1
[1] "run"       "4"         "completed"

Synthesis
-----------
 age workclass fnlwgt education marital_status occupation relationship race sex capital_gain
 capital_loss hours_per_week nat

In [41]:
cps_res <- simulation(data = cpspop, nrun = 10, outer_folds = 5, inner_folds = 5, mtry_steps = 10, ntree_steps = 10, cp_steps = 10)


Synthesis
-----------
 tax income csp age educ marital race sex ss
[1] 4
[1] "run"       "1"         "completed"

Synthesis
-----------
 tax income csp age educ marital race sex ss
[1] 4
[1] "run"       "2"         "completed"


In [63]:
#Saving the data:
saveRDS(cps_res, file = (paste0(directory, "/simulation/cps_CART_res.RData")))
saveRDS(adult_res, file = (paste0(directory, "/simulation/adult_CART_res.RData")))

Warning message in gzfile(file, mode):
"kann komprimierte Datei '/Users/emmafoessing/Documents/Master/MA/Code/Master-Thesis/simulation/cps_CART_res.RData' nicht "offnen. Grund evtl. 'No such file or directory'"


ERROR: Error in gzfile(file, mode): kann Verbindung nicht "offnen
